In [1]:
import pandas as pd

# Definir variable global para el dataset
global df_global
df_global = None

# Función para asignar dataset ya cargado a df_global (aquí cargas tu archivo manualmente o con otro método)
def asignar_dataset(df):
    global df_global
    df_global = df.copy()
    print("Dataset asignado correctamente. Primeras filas:")
    display(df_global.head())


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, precision_score, accuracy_score

paleta = ['#328E6E', '#67AE6E', '#90C67C', '#E1EEBC']

sns.set_style("whitegrid")
plt.rcParams['figure.facecolor'] = 'white'

def preparar_datos(df):
    X = df[['ProductCategory', 'ProductBrand', 'ProductPrice',
            'CustomerAge', 'CustomerGender', 'PurchaseFrequency',
            'CustomerSatisfaction']].copy()
    y = df['PurchaseIntent']
    X_encoded = pd.get_dummies(X, columns=['ProductCategory', 'ProductBrand'], drop_first=True)
    return train_test_split(X_encoded, y, test_size=0.2, random_state=42)

def graficos_comunes(df):
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 2, 1)
    df['CustomerGender_cat'] = df['CustomerGender'].map({0: 'Femenino', 1: 'Masculino'})
    sns.stripplot(data=df, x='CustomerGender_cat', y='CustomerSatisfaction',
                  hue='PurchaseIntent', dodge=True, jitter=0.25, alpha=0.7,
                  palette={0: paleta[0], 1: paleta[1]})
    plt.title('Genero vs Satisfacción por Compra')
    plt.xlabel('Genero del Cliente')
    plt.ylabel('Satisfacción (1-5)')
    plt.legend(title='Compra', labels=['No', 'Si'])

    plt.subplot(1, 2, 2)
    sns.stripplot(data=df, x='CustomerGender_cat', y='CustomerAge',
                 hue='PurchaseIntent', dodge=True, jitter=0.2, alpha=0.5,
                 palette={0: paleta[0], 1: paleta[1]})
    plt.title('Edad vs Genero por Compra')
    plt.xlabel('Genero del Cliente')
    plt.ylabel('Edad')
    handles, labels = plt.gca().get_legend_handles_labels()
    plt.legend(handles[:2], ['No', 'Si'], title='Compra', loc='upper right')
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 6))
    cross_tab = pd.crosstab(index=df['CustomerGender'], columns=df['PurchaseIntent'], normalize='index') * 100
    cross_tab.plot(kind='bar', stacked=True, color=[paleta[0], paleta[1]], edgecolor='black')
    plt.title('Proporción de Compras por Género')
    plt.xlabel('Genero (0=Femenino, 1=Masculino)')
    plt.ylabel('Porcentaje (%)')
    plt.xticks(rotation=0)
    plt.legend(['No compró', 'Compró'], title='Decision')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

def entrenar_y_evaluar_modelo(modelo_seleccionado, df):
    X_train, X_test, y_train, y_test = preparar_datos(df)
    if modelo_seleccionado == 'Regresión Logística':
        model = LogisticRegression(max_iter=1000)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        probs = model.predict_proba(X_test)[:, 1]

    elif modelo_seleccionado == 'KNN':
        k_values = list(range(19, 41))
        accuracies = []
        for k in k_values:
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)
            y_pred_k = knn.predict(X_test)
            accuracies.append(accuracy_score(y_test, y_pred_k))
        best_k = k_values[np.argmax(accuracies)]
        print(f"Mejor K encontrado: {best_k} con accuracy={max(accuracies):.4f}")
        model = KNeighborsClassifier(n_neighbors=best_k)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        if hasattr(model, "predict_proba"):
            probs = model.predict_proba(X_test)[:, 1]
        else:
            probs = np.array([np.nan]*len(y_test))

    else:  # Árbol de Decisión
        model = DecisionTreeClassifier(random_state=42, max_depth=5)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        if hasattr(model, "predict_proba"):
            probs = model.predict_proba(X_test)[:, 1]
        else:
            probs = np.array([np.nan]*len(y_test))

    # Mostrar resultados y gráficos igual que antes (puedes copiar el código de la interfaz)
    return model, X_test, y_test, y_pred, probs


In [11]:
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, precision_score, accuracy_score
from ipywidgets import Dropdown, Button, Output, VBox, HBox, Layout, ToggleButtons

# Paleta personalizada
paleta = ['#328E6E', '#67AE6E', '#90C67C', '#E1EEBC']

sns.set_style("whitegrid")
plt.rcParams['figure.facecolor'] = paleta[3]

df_global = None
modelo_global = None
resultados_global = {}

btn_cargar = Button(description="Cargar Dataset", layout=Layout(width='160px'), style={'font_weight': 'bold'})
dropdown_modelo = Dropdown(options=[
    ('Regresión Logística', 'logistic'),
    ('KNN (K-Vecinos)', 'knn'),
    ('Árbol de Decisión', 'tree')
], description='Modelo:', layout=Layout(width='230px'), style={'description_width': 'initial', 'font_weight': 'bold'})

btn_ejecutar = Button(description="Ejecutar Modelo", layout=Layout(width='160px'), style={'font_weight': 'bold'})
btn_ejecutar.disabled = True

output_info = Output(layout=Layout(border=f'2px solid {paleta[1]}', border_radius='12px', padding='10px', height='120px', overflow='auto', margin='10px 0', background_color=paleta[3]))
output_reporte = Output(layout=Layout(border=f'2px solid {paleta[2]}', border_radius='12px', padding='10px', height='140px', overflow='auto', margin='10px 0', background_color=paleta[3]))
output_grafico = Output(layout=Layout(border=f'2px solid {paleta[0]}', border_radius='12px', padding='10px', height='520px', overflow='auto', margin='10px 0', background_color='#f9fff7'))

# Botones para seleccionar gráfico a mostrar
toggle_graficos = ToggleButtons(
    options=[
        ('Gráfico 1: Satisfacción por género', 'graf1'),
        ('Gráfico 2: Edad por género', 'graf2'),
        ('Gráfico 3: Compra por género', 'graf3'),
        ('Matriz de Confusión', 'confusion'),
        ('Probabilidad compra x Producto+Marca', 'prob'),
        ('Comparación Predicción vs Real', 'comp')
    ],
    description='Visualizar:',
    disabled=True,
    button_style='',
    tooltips=[
        'Satisfacción cliente según género y compra',
        'Edad cliente según género y compra',
        'Porcentaje de compra según género',
        'Matriz de Confusión del modelo',
        'Probabilidad promedio de compra por producto y marca',
        'Comparación entre predicción y compra real'
    ],
    style={'description_width': 'initial'},
    layout=Layout(width='100%', margin='10px 0')
)

# Funciones
def asignar_dataset(df):
    global df_global
    df_global = df.copy()

def mostrar_info_dataset():
    with output_info:
        output_info.clear_output()
        print(f"Dataset cargado con {df_global.shape[0]} filas y {df_global.shape[1]} columnas.\n")
        print("Tipos de datos:")
        print(df_global.dtypes)

def preparar_datos(df):
    X = df[['ProductCategory', 'ProductBrand', 'ProductPrice',
            'CustomerAge', 'CustomerGender', 'PurchaseFrequency',
            'CustomerSatisfaction']].copy()
    y = df['PurchaseIntent']
    X_encoded = pd.get_dummies(X, columns=['ProductCategory', 'ProductBrand'], drop_first=True)
    return train_test_split(X_encoded, y, test_size=0.2, random_state=42)

def entrenar_modelo(modelo, df):
    global resultados_global
    X_train, X_test, y_train, y_test = preparar_datos(df)

    if modelo == 'logistic':
        model = LogisticRegression(max_iter=1000)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        probs = model.predict_proba(X_test)[:, 1]
    elif modelo == 'knn':
        k_values = list(range(19, 41))
        accuracies = []
        for k in k_values:
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)
            y_pred_k = knn.predict(X_test)
            accuracies.append(accuracy_score(y_test, y_pred_k))
        best_k = k_values[np.argmax(accuracies)]
        with output_reporte:
            print(f"Mejor K encontrado: {best_k} con accuracy={max(accuracies):.4f}")
        model = KNeighborsClassifier(n_neighbors=best_k)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        if hasattr(model, "predict_proba"):
            probs = model.predict_proba(X_test)[:, 1]
        else:
            probs = np.array([np.nan]*len(y_test))
    else:
        model = DecisionTreeClassifier(random_state=42, max_depth=5)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        if hasattr(model, "predict_proba"):
            probs = model.predict_proba(X_test)[:, 1]
        else:
            probs = np.array([np.nan]*len(y_test))

    # Guardamos resultados globales para usar en botones
    resultados_global = {
        'model': model,
        'X_test': X_test,
        'y_test': y_test,
        'y_pred': y_pred,
        'probs': probs,
        'df': df,
    }
    return resultados_global

def mostrar_reporte():
    y_test = resultados_global['y_test']
    y_pred = resultados_global['y_pred']
    reporte = classification_report(y_test, y_pred, target_names=['No Compra', 'Compra'])
    precision = precision_score(y_test, y_pred)
    with output_reporte:
        output_reporte.clear_output()
        print("Reporte de Clasificación:\n")
        print(reporte)
        print(f"Precisión del modelo: {precision:.2f}")

def plot_grafico(seleccion):
    df = resultados_global['df']
    y_test = resultados_global['y_test']
    y_pred = resultados_global['y_pred']
    probs = resultados_global['probs']
    X_test = resultados_global['X_test']

    with output_grafico:
        output_grafico.clear_output()
        if seleccion == 'graf1':
            plt.figure(figsize=(14, 6))
            df['CustomerGender_cat'] = df['CustomerGender'].map({0: 'Femenino', 1: 'Masculino'})
            sns.stripplot(data=df, x='CustomerGender_cat', y='CustomerSatisfaction',
                          hue='PurchaseIntent', dodge=True, jitter=0.25, alpha=0.8,
                          palette={0: paleta[0], 1: paleta[1]})
            plt.title('Genero vs Satisfacción')
            plt.xlabel('Género')
            plt.ylabel('Satisfacción (1-5)')
            plt.legend(title='Compra', labels=['No', 'Sí'])
            plt.tight_layout(pad=3)
            plt.show()

        elif seleccion == 'graf2':
            plt.figure(figsize=(14, 6))
            df['CustomerGender_cat'] = df['CustomerGender'].map({0: 'Femenino', 1: 'Masculino'})
            sns.stripplot(data=df, x='CustomerGender_cat', y='CustomerAge',
                          hue='PurchaseIntent', dodge=True, jitter=0.2, alpha=0.6,
                          palette={0: paleta[0], 1: paleta[1]})
            plt.title('Edad vs Género')
            plt.xlabel('Género')
            plt.ylabel('Edad')
            handles, labels = plt.gca().get_legend_handles_labels()
            plt.legend(handles[:2], ['No', 'Sí'], title='Compra', loc='upper right')
            plt.tight_layout(pad=3)
            plt.show()

        elif seleccion == 'graf3':
            plt.figure(figsize=(14, 7))
            cross_tab = pd.crosstab(df['CustomerGender'], df['PurchaseIntent'], normalize='index') * 100
            cross_tab.plot(kind='bar', stacked=True, color=[paleta[0], paleta[1]], edgecolor='black')
            plt.title('Proporción de Compra por Género')
            plt.xlabel('Género (0=Femenino, 1=Masculino)')
            plt.ylabel('Porcentaje (%)')
            plt.xticks(rotation=0)
            plt.legend(['No compró', 'Compró'], title='Compra')
            plt.grid(axis='y', alpha=0.3)
            plt.tight_layout(pad=3)
            plt.show()

        elif seleccion == 'confusion':
            cm = confusion_matrix(y_test, y_pred)
            disp = ConfusionMatrixDisplay(cm, display_labels=['No Compra', 'Compra'])
            fig, ax = plt.subplots(figsize=(7,7))
            disp.plot(cmap=plt.cm.Greens, ax=ax)
            ax.set_xlabel('Predicción')
            ax.set_ylabel('Real')
            ax.grid(False)
            plt.title('Matriz de Confusión')
            plt.tight_layout(pad=3)
            plt.show()

        elif seleccion == 'prob':
            if np.isnan(probs).all():
                print("El modelo no provee probabilidades para graficar.")
            else:
                results = df.loc[X_test.index, ['ProductCategory', 'ProductBrand', 'ProductPrice']].copy()
                results['ProbabilidadCompra'] = probs
                results['PrediccionCompra'] = y_pred
                results['CompraReal'] = y_test.values
                results['ProductoMarca'] = results['ProductCategory'] + " " + results['ProductBrand']

                prob_promedio = results.groupby('ProductoMarca')['ProbabilidadCompra'].mean().sort_values(ascending=False)
                plt.figure(figsize=(16, 8))
                prob_promedio.plot(kind='bar', color=paleta[2])
                plt.ylabel('Probabilidad Promedio de Compra')
                plt.xlabel('Producto + Marca')
                plt.title('Probabilidad Promedio de Compra por Producto y Marca')
                plt.xticks(rotation=45, ha='right')
                plt.ylim(0, 1)
                plt.grid(axis='y', alpha=0.3)
                plt.tight_layout(pad=3)
                plt.show()

        elif seleccion == 'comp':
            if np.isnan(probs).all():
                print("El modelo no provee probabilidades para graficar.")
            else:
                results = df.loc[X_test.index, ['ProductCategory', 'ProductBrand', 'ProductPrice']].copy()
                results['ProbabilidadCompra'] = probs
                results['PrediccionCompra'] = y_pred
                results['CompraReal'] = y_test.values
                results['ProductoMarca'] = results['ProductCategory'] + " " + results['ProductBrand']

                compra_real_promedio = results.groupby('ProductoMarca')['CompraReal'].mean()
                prob_promedio = results.groupby('ProductoMarca')['ProbabilidadCompra'].mean()

                df_plot = pd.DataFrame({
                    'Predicción': prob_promedio,
                    'Proporción Real': compra_real_promedio
                }).sort_values(by='Predicción', ascending=False)

                n = len(df_plot)
                ind = np.arange(n)
                width = 0.35

                plt.figure(figsize=(18, 8))
                plt.bar(ind - width/2, df_plot['Predicción'], width, label='Predicción', alpha=0.7, color=paleta[1])
                plt.bar(ind + width/2, df_plot['Proporción Real'], width, label='Proporción Real de Compra', alpha=0.7, color=paleta[3])
                plt.ylabel('Promedio de Compra')
                plt.title('Comparación Predicción vs Proporción Real')
                plt.xticks(ind, df_plot.index, rotation=45, ha='right')
                plt.ylim(0, 1)
                plt.legend()
                plt.grid(axis='y', alpha=0.3)
                plt.tight_layout(pad=3)
                plt.show()

def on_cargar_click(b):
    global df_global, modelo_global, resultados_global
    resultados_global = {}
    modelo_global = None
    with output_reporte:
        output_reporte.clear_output()
    with output_grafico:
        output_grafico.clear_output()
    with output_info:
        output_info.clear_output()
    subir_archivo(b)

def on_ejecutar_click(b):
    global df_global, modelo_global, resultados_global
    if df_global is None:
        with output_reporte:
            output_reporte.clear_output()
            print("Primero debes cargar un dataset.")
        return
    output_reporte.clear_output()
    output_grafico.clear_output()
    modelo_global = dropdown_modelo.value
    resultados_global = entrenar_modelo(modelo_global, df_global)
    mostrar_reporte()
    toggle_graficos.disabled = False
    toggle_graficos.value = 'graf1'  # Mostrar gráfico 1 por defecto
    plot_grafico('graf1')

def on_toggle_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        plot_grafico(change['new'])

btn_cargar.on_click(on_cargar_click)
btn_ejecutar.on_click(on_ejecutar_click)
toggle_graficos.observe(on_toggle_change)

# Layout

controls = HBox([btn_cargar, dropdown_modelo, btn_ejecutar], layout=Layout(
    justify_content='space-around',
    align_items='center',
    padding='10px',
    margin='10px 0',
    border=f'2px solid {paleta[0]}',
    border_radius='20px',
    background_color=paleta[3],
    box_shadow='3px 3px 10px rgba(50, 142, 110, 0.15)'
))

panel = VBox([
    controls,
    info_dataset,
    output_reporte,
    toggle_graficos,
    output_grafico,
], layout=Layout(width='95%', max_width='980px', margin='20px auto'))

display(panel)



Saving consumer_electronics_sales_data.csv to consumer_electronics_sales_data (6).csv
